In [1]:
import pandas as pd
import numpy as np
import torch.nn as nn
from guizero import *
import PIL
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True #imagenes ligeramente corruptas OK
from transformers import AutoImageProcessor
from transformers import AutoModelForImageClassification

In [2]:
#datos

In [3]:
data=None
try:
    data=pd.read_feather("eval_set.feather")
except:
    data=pd.read_feather("eval_set.bak.feather")

In [4]:
data.describe()

,Path,auto_labels,labels
count,270,270,270
unique,270,1,1
top,images/2024/image_1.jpg,,
freq,1,270,270


In [5]:
# utils

In [6]:
def load_image(path):
    return PIL.Image.open(path)

In [7]:
def save(backup):
    data.to_feather(f"eval_set{'.bak' if backup else ''}.feather")

In [8]:
def get_next(col="labels"):
    f=data[data[col]==""]
    if(f.shape[0]==0):
        return None
    return f.iloc[0]

In [9]:
def set_label(idx,label,col="labels"):
    data.iloc[idx][col]=label

In [10]:
# classifier

In [17]:
model_name="MimModels/swinv2-base-patch4-window16-256-popo"

In [18]:
device="cuda"

In [19]:
model = AutoModelForImageClassification.from_pretrained(model_name).to(device)

In [20]:
image_processor = AutoImageProcessor.from_pretrained(model_name)

In [21]:
def process(image):
    return image_processor(images=image, return_tensors="pt").to(model.device)["pixel_values"]

In [22]:
def consume(pixels):
    logits=model(pixel_values=pixels).logits
    return nn.functional.softmax(logits, dim=-1).detach().cpu().numpy().flatten()

In [23]:
#p=consume(process(load_image(data.iloc[0]["Path"])))
#p

In [24]:
def classify(probabilities):
    idx=probabilities.argmax(-1).item()
    return model.config.id2label[idx]

In [25]:
#model.config.id2label

In [26]:
#model.config.id2label[4],data.iloc[0]["y"]

In [27]:
while(True):
    r_count=data[data["auto_labels"]==""].shape[0]
    print(f"Remaining: {r_count} ({r_count/data.shape[0]*100:.02f}%)",end="\r")
    nxt=get_next(col="auto_labels")
    if nxt is None:
        break
    try:
        probs=consume(process(load_image(nxt["Path"])))
        cl=classify(probs)
    except:
        cl="UNK"
    save(True)
    data.loc[nxt.name,"auto_labels"]=cl
    save(False)
    

In [28]:
#data["ny"]=np.where(data["y"]!=data["my"],"",data["y"])

In [29]:
#save(True)

In [30]:
#save(False)

In [31]:
# initial state

In [32]:
r_count=data[data["labels"]==""].shape[0]
m_count=r_count
nxt=get_next()
if nxt is None:
    raise "You're done, go home!"

In [33]:
# interface

In [34]:
app = App(bg="black")
app.set_full_screen()
w,h=app.width,app.height
r_count=data[data["labels"]==""].shape[0]
current_img=load_image(nxt["Path"])
scaled_w=current_img.size[0]/current_img.size[1]*h*.6
pic=Picture(app, image=current_img, width=int(scaled_w), height=int(h*.6))
progress=Text(app, text=f"Remaining: {r_count} ({100-r_count/m_count*100:.02f}% complete)", width="fill", height=1,  )
progress.text_size*=3
progress.text_color="white"
info=Text(app, text=f"Model says: {nxt['auto_labels']}", width="fill", height=1,  )
info.text_size*=3
info.text_color="white"
bottom=Box(app, layout="grid", width="fill", height="fill")
buttons=None

def buttonPressed(i):
    global r_count, nxt, current_img
    print(model.config.id2label[i],"pressed")
    save(True)
    data.loc[nxt.name,"labels"]=model.config.id2label[i]
    save(False)

    r_count=data[data["labels"]==""].shape[0]
    nxt=get_next(col="labels")
    progress.value=f"Remaining: {r_count} ({100-r_count/m_count*100:.02f}% complete)"
    info.value=f"Model says: {nxt['auto_labels']}"
    current_img=load_image(nxt["Path"])
    pic.image=current_img
    scaled_w=current_img.size[0]/current_img.size[1]*h*.7
    pic.resize(int(scaled_w), int(h*.8))
    
def makeButton(i):
    label=model.config.id2label[i]
    b=PushButton(bottom, text=label, grid=[i,0], padx=0, pady=0, width=w//int(5*20), height=2, command=buttonPressed, args=[i])
    b.bg="white"
    b.text_size*=3
    return b
buttons=[makeButton(i) for i in range(5)]

app.focus()
app.display()

EXP+FUM pressed
UNK pressed
UNK pressed
UNK pressed
FUM pressed
INA pressed
INA pressed
EXP+FUM pressed
EXP+FUM pressed
EXP+FUM pressed
EXP+FUM pressed
FUM pressed
FUM pressed
FUM pressed
EXP+FUM pressed
EXP+FUM pressed
UNK pressed
FUM pressed
EXP+FUM pressed
FUM pressed
EXP+FUM pressed
INA pressed
EXP+FUM pressed
FUM pressed
FUM pressed
EXP+FUM pressed
UNK pressed
EXP+FUM pressed
FUM pressed
UNK pressed
EXP+FUM pressed
EXP+FUM pressed
EXP+FUM pressed
FUM pressed
EXP+FUM pressed
FUM pressed
FUM pressed
EXP+FUM pressed
EXP+FUM pressed
EXP+FUM pressed
FUM pressed
EXP+FUM pressed
EXP+FUM pressed
EXP+FUM pressed
EXP+FUM pressed
FUM pressed
FUM pressed
FUM pressed
EXP+FUM pressed
EXP+FUM pressed
UNK pressed
FUM pressed
FUM pressed
FUM pressed
FUM pressed
EXP+FUM pressed
UNK pressed
FUM pressed
FUM pressed
EXP+FUM pressed
EXP+FUM pressed
UNK pressed
FUM pressed
UNK pressed
FUM pressed
INA pressed
EXP+FUM pressed
FUM pressed
EXP+FUM pressed
INA pressed
EXP+FUM pressed
FUM pressed
EXP+FUM pres

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\alexi\mambaforge\lib\tkinter\__init__.py", line 1921, in __call__
    return self.func(*args)
  File "C:\Users\alexi\mambaforge\lib\site-packages\guizero\PushButton.py", line 206, in _command_callback
    self._command()
  File "C:\Users\alexi\mambaforge\lib\site-packages\guizero\utilities.py", line 364, in <lambda>
    return lambda: func_name(*args)
  File "C:\Users\alexi\AppData\Local\Temp\ipykernel_2120\4194483799.py", line 22, in buttonPressed
    save(False)
  File "C:\Users\alexi\AppData\Local\Temp\ipykernel_2120\4216359964.py", line 2, in save
    data.to_feather(f"eval_set{'.bak' if backup else ''}.feather")
  File "C:\Users\alexi\mambaforge\lib\site-packages\pandas\core\frame.py", line 2714, in to_feather
    to_feather(self, path, **kwargs)
  File "C:\Users\alexi\mambaforge\lib\site-packages\pandas\io\feather_format.py", line 93, in to_feather
    with get_handle(
  File "C:\Users\alexi\mambafo

FUM pressed
UNK pressed
EXP+FUM pressed
EXP+FUM pressed
UNK pressed
UNK pressed
FUM pressed
EXP+FUM pressed
EXP+FUM pressed
FUM pressed
EXP+FUM pressed
EXP+FUM pressed
EXP+FUM pressed
EXP+FUM pressed
EXP+FUM pressed
FUM pressed
UNK pressed
EXP+FUM pressed
EXP+FUM pressed
UNK pressed
EXP+FUM pressed
EXP+FUM pressed
UNK pressed
EXP+FUM pressed
EXP+FUM pressed
EXP+FUM pressed
FUM pressed
FUM pressed
FUM pressed
EXP+FUM pressed
EXP+FUM pressed
EXP+FUM pressed
FUM pressed
FUM pressed
EXP+FUM pressed
EXP+FUM pressed
EXP+FUM pressed
EXP+FUM pressed
EXP+FUM pressed
EXP+FUM pressed
EXP+FUM pressed
EXP+FUM pressed
EXP+FUM pressed
EXP+FUM pressed
FUM pressed
EXP+FUM pressed
EXP+FUM pressed
EXP+FUM pressed
INA pressed
FUM pressed
UNK pressed
FUM pressed
EXP+FUM pressed
UNK pressed
FUM pressed
FUM pressed
FUM pressed
EXP+FUM pressed
EXP+FUM pressed
UNK pressed
INA pressed
FUM pressed
FUM pressed
FUM pressed
EXP+FUM pressed
FUM pressed
EXP+FUM pressed
FUM pressed
FUM pressed
FUM pressed
FUM pressed


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\alexi\mambaforge\lib\tkinter\__init__.py", line 1921, in __call__
    return self.func(*args)
  File "C:\Users\alexi\mambaforge\lib\site-packages\guizero\PushButton.py", line 206, in _command_callback
    self._command()
  File "C:\Users\alexi\mambaforge\lib\site-packages\guizero\utilities.py", line 364, in <lambda>
    return lambda: func_name(*args)
  File "C:\Users\alexi\AppData\Local\Temp\ipykernel_2120\4194483799.py", line 27, in buttonPressed
    info.value=f"Model says: {nxt['auto_labels']}"
TypeError: 'NoneType' object is not subscriptable


FUM pressed


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\alexi\mambaforge\lib\tkinter\__init__.py", line 1921, in __call__
    return self.func(*args)
  File "C:\Users\alexi\mambaforge\lib\site-packages\guizero\PushButton.py", line 206, in _command_callback
    self._command()
  File "C:\Users\alexi\mambaforge\lib\site-packages\guizero\utilities.py", line 364, in <lambda>
    return lambda: func_name(*args)
  File "C:\Users\alexi\AppData\Local\Temp\ipykernel_2120\4194483799.py", line 21, in buttonPressed
    data.loc[nxt.name,"labels"]=model.config.id2label[i]
AttributeError: 'NoneType' object has no attribute 'name'


FUM pressed


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\alexi\mambaforge\lib\tkinter\__init__.py", line 1921, in __call__
    return self.func(*args)
  File "C:\Users\alexi\mambaforge\lib\site-packages\guizero\PushButton.py", line 206, in _command_callback
    self._command()
  File "C:\Users\alexi\mambaforge\lib\site-packages\guizero\utilities.py", line 364, in <lambda>
    return lambda: func_name(*args)
  File "C:\Users\alexi\AppData\Local\Temp\ipykernel_2120\4194483799.py", line 21, in buttonPressed
    data.loc[nxt.name,"labels"]=model.config.id2label[i]
AttributeError: 'NoneType' object has no attribute 'name'


FUM pressed


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\alexi\mambaforge\lib\tkinter\__init__.py", line 1921, in __call__
    return self.func(*args)
  File "C:\Users\alexi\mambaforge\lib\site-packages\guizero\PushButton.py", line 206, in _command_callback
    self._command()
  File "C:\Users\alexi\mambaforge\lib\site-packages\guizero\utilities.py", line 364, in <lambda>
    return lambda: func_name(*args)
  File "C:\Users\alexi\AppData\Local\Temp\ipykernel_2120\4194483799.py", line 21, in buttonPressed
    data.loc[nxt.name,"labels"]=model.config.id2label[i]
AttributeError: 'NoneType' object has no attribute 'name'


In [38]:
data["labels"].value_counts()

labels
EXP+FUM    128
FUM         88
UNK         43
INA         10
EXP          1
Name: count, dtype: int64

In [40]:
save(False)